In [1]:
#Working with VizWiz dataset
import os
import json
import requests

base_url = 'https://ivc.ischool.utexas.edu/VizWiz/data'
img_dir = '%s/Images/' %base_url

split_train = 'train'
split_val = 'val'
split_test = 'test'

trainFile = '%s/Annotations/%s.json' %(base_url, split_train)
trainData = requests.get(trainFile, allow_redirects = True)
valFile = '%s/Annotations/%s.json' %(base_url, split_val)
valData = requests.get(valFile, allow_redirects = True)
testFile = '%s/Annotations/%s.json' %(base_url, split_test)
testData = requests.get(valFile, allow_redirects = True)

In [2]:
trainingData = trainData.json()
validationData = valData.json()
testingData = testData.json()

listQuestion = []
listLabel = []
for vq in trainingData[0:800]:
    question = vq['question']
    label = vq['answerable']
    listQuestion.append(question)
    listLabel.append(label)

listValQuestion = []
listValLabel = []
for vq in validationData[0:300]:
    question_val = vq['question']
    label_val = vq['answerable']
    listValQuestion.append(question_val)
    listValLabel.append(label_val)

listTestQuestion = []
for vq in testingData[0:100]:
    question_test = vq['question']
    listTestQuestion.append(question_test)

In [ ]:
from sklearn.feature_extraction.text import 
allQuestions = listQuestion + listValQuestion + listTestQuestion



In [ ]:
#Retrive file from URL and store it locally
split = 'train'
trainFile = '%s/Annotations/%s.json' %(base_url, split_train)
trainData = requests.get(trainFile, allow_redirects = True)
print(trainFile)

In [ ]:
#Read the local file
numTrainVQs = 4
trainingData = trainData.json()
for vq in trainingData[0:numTrainVQs]:
    image_name = vq['image']
    question = vq['question']
    label = vq['answerable']
    print(image_name)
    print(question)
    print(label)

In [ ]:
split = 'val'
valFile = '%s/Annotations/%s.json' %(base_url, split)
valData = requests.get(valFile, allow_redirects = True)
print(valFile)

In [ ]:
numValVQs = 3
validationData = valData.json()
for vq in validationData[0:numValVQs]:
    image_name = vq['image']
    question = vq['question']
    label = vq['answerable']
    print(image_name)
    print(question)
    print(label)

In [3]:
#Extract features to describe the questions
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def extract_language_features(question):
    #Preprocessing questions
    question = question.lower()
    
    words = question.split()
    num_unique_words = len(set(words))
    num_words = len(words)
    rate = num_unique_words/num_words
    
    partsOfSpeechTag = nltk.pos_tag(words)
    tag_fd = nltk.FreqDist(partsOfSpeechTag)
    determinerCount = tag_fd['DT']
    verbCount = tag_fd['VB']
    
    #tokens=nltk.word_tokenize(words)
    fdist = FreqDist(words)

    #tokenized_question
    
    featureVector = [determinerCount, verbCount]
    return featureVector

numVQs = 4
for vq in trainingData[0:numVQs]:
    #Question features
    question = vq['question']
    print(question)
    questionFeature = extract_language_features(question)
    print(questionFeature)
    
    image_name = vq['image']
    image_url = img_dir + image_name
    print(image_url, '\n')

What's the name of this product?
[0, 0]
https://ivc.ischool.utexas.edu/VizWiz/data/Images/VizWiz_train_000000000000.jpg 

Can you tell me what is in this can please?
[0, 0]
https://ivc.ischool.utexas.edu/VizWiz/data/Images/VizWiz_train_000000000001.jpg 

Is this enchilada sauce or is this tomatoes?  Thank you.
[0, 0]
https://ivc.ischool.utexas.edu/VizWiz/data/Images/VizWiz_train_000000000002.jpg 

What is the captcha on this screenshot?
[0, 0]
https://ivc.ischool.utexas.edu/VizWiz/data/Images/VizWiz_train_000000000003.jpg 



In [4]:
from skimage import io
from skimage.transform import resize
from skimage import color
from skimage import feature
import numpy as np

def extrac_image_features(image_url):
    #Read image
    image = io.imread(image_url)
    
    #Preprocess image
    width = 255
    height = 255
    image = resize(image, (width, height))#Ensuring all images have the same dimension
    greyscale_image = color.rgb2gray(image)#Restricting the dimension of our data from 3D to 2D
    
    #Extract features
    descs, descs_img = feature.daisy(greyscale_image, step=180, radius=58, rings=2, histograms=6,
                         orientations=8, visualize=True)
    edges1 = feature.canny(greyscale_image)
    edges2 = feature.canny(greyscale_image, sigma=0.3)
    descs = descs.ravel()
    edges1 = edges1.ravel()
    edges2 = edges2.ravel()
    featureVector = np.concatenate((descs, edges1, edges2))
    return featureVector

numVQs = 4
for vq in trainingData[0:numVQs]:
    image_name = vq['image']
    image_url = img_dir + image_name
    #print(image_url)
    image_feature = extrac_image_features(image_url)
    print(image_feature)

/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


[0.00779634 0.00823141 0.00718225 ... 0.         0.         0.        ]
[0.0049621  0.00782353 0.0087966  ... 0.         0.         0.        ]
[0.00860813 0.01153974 0.00972141 ... 0.         0.         0.        ]
[0.0016703  0.00336014 0.0061584  ... 0.         0.         0.        ]


In [5]:
import numpy as np
import pandas as pd
import operator
from functools import reduce

X_train = []
X_val = []
X_test = []
y_train = []
y_val = []

In [6]:
for vq in trainingData[0:800]:
    # Question features
    question = vq['question']
    question_feature = extract_language_features(question)
    question_feature = np.array(question_feature)
    
    #Image features
    image_name = vq['image']
    image_url = img_dir + image_name
    image_feature = extrac_image_features(image_url)
    #print(np.array(image_feature).shape)
       
    #Concatenate the question and image features
    multimodal_features = np.concatenate((question_feature, image_feature), axis = None)
    #features = np.append(features, np.array(multimodal_features), axis = 1)
    X_train.append(multimodal_features)
    #print(multimodal_features)
    #print(multimodal_features.shape)
    
    #Exreact labels
    label = vq['answerable']
    y_train.append(label)
print('Done')
#print(features)
#print(reduce(operator.add, features))
#print(labels)

/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Done


In [7]:
for vq in validationData[0:300]:
    # Question features
    question = vq['question']
    question_feature = extract_language_features(question)
    question_feature = np.array(question_feature)
    
    #Image features
    image_name = vq['image']
    image_url = img_dir + image_name
    image_feature = extrac_image_features(image_url)
       
    #Concatenate the question and image features
    multimodal_features = np.concatenate((question_feature, image_feature), axis = None)
    X_val.append(multimodal_features)
    
    #Exreact labels
    label = vq['answerable']
    y_val.append(label)
print('Done')

/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Done


In [8]:
for vq in testingData[100:200]:
    # Question features
    question = vq['question']
    question_feature = extract_language_features(question)
    question_feature = np.array(question_feature)
    
    #Image features
    image_name = vq['image']
    image_url = img_dir + image_name
    image_feature = extrac_image_features(image_url)
       
    #Concatenate the question and image features
    multimodal_features = np.concatenate((question_feature, image_feature), axis = None)
    X_test.append(multimodal_features)

print('Done')

/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/anaconda3/lib/python3.7/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Done


In [9]:
y_train = np.transpose(y_train)
y_val = np.transpose(y_val)

np.save('X_train.npy', X_train)
np.save('y_train.npy', y_train)
np.save('X_val.npy', X_val)
np.save('y_val.npy', y_val)
np.save('X_test.npy', X_test)

In [10]:
X_train = np.load('X_train.npy')
X_val = np.load('X_val.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_val = np.load('y_val.npy')

In [11]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)

(800, 130156)
(300, 130156)
(100, 130156)
(800,)
(300,)


In [30]:
from sklearn.decomposition import PCA

pca = PCA(n_components=7)
pca.fit(X_train)

#cum_var = np.cumsum(pca.explained_variance_ratio_)
#nPC = np.min(np.where(cum_var >= 0.90)) + 1

X_train_reduced = pca.transform(X_train)
X_val_reduced = pca.transform(X_val)
X_test_reduced = pca.transform(X_test)

In [31]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

gaussian_clf = GaussianNB()
gaussian_clf.fit(X_train_reduced, y_train)
val_pred = gaussian_clf.predict(X_val_reduced)
val_accuracy = accuracy_score(y_val, val_pred)
val_accuracy

0.6966666666666667

In [33]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression()
lr_clf.fit(X_train_reduced, y_train)
val_pred_1 = lr_clf.predict(X_val_reduced)
val_accuracy_1 = accuracy_score(y_val, val_pred)
val_accuracy_1

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6966666666666667

In [42]:
from sklearn.svm import SVC

svm_clf = SVC(C=7, degree=2, gamma=0.1, kernel='rbf')
svm_clf.fit(X_train_reduced, y_train)
val_pred_2 = svm_clf.predict(X_val_reduced)
val_accuracy_2 = accuracy_score(y_val, val_pred)
val_accuracy_2

0.6966666666666667

In [36]:
y_test = gaussian_clf.predict(X_test_reduced)
print(y_test)
print(type(y_test))

[1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1]
<class 'numpy.ndarray'>


In [37]:
np.savetxt('Prediction.csv',y_test,delimiter=',')

In [40]:
import pandas as pd

pd_data = pd.DataFrame(y_test, columns=['Prediction'])
#print(pd_data)
pd_data.to_csv('prediction.csv', index=False)